# XGBoost
TODO:
Try normalise, z_score, remove redundant features

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# General
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ML
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE

import xgboost as xgb

# Custom
import sys,os
# %cd /content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution
# sys.path.append('/content/fuck/My Drive/ETHZ/Autumn2020/AML/Git/AML/Task_1/Solution')
sys.path.append('/home/jovyan/work/AML/Task_1/Solution') # I would like a cleaner solution but works for now
# import Components.Imputation as Imputation
import Components.Outlier_Detection_1D as Outlier_Detection_1D
import Components.Outlier_Detection_ND as Outlier_Detection_ND
# import Components.Feature_Selection as Feature_Selection
import Components.Normalisation as Normalisation
import Components.data_fetching as data_fetching

# # CAREFUL:
# # If you make changes to a custom module, you have to reload it, i.e rerun this cell
import importlib
# importlib.reload(Imputation)
importlib.reload(Outlier_Detection_1D)
importlib.reload(Outlier_Detection_ND)
# importlib.reload(Feature_Selection)
importlib.reload(Normalisation)
importlib.reload(data_fetching)

<module 'Components.data_fetching' from '/home/jovyan/work/AML/Task_1/Solution/Components/data_fetching.py'>

## Pipeline Optimisatoin

In [ ]:
X,y = data_fetching.get_train_data()
x_test_final = data_fetching.get_test_data()

X = Normalisation.gaussian(X,n_quants=50)
X = Outlier_Detection_1D.z_score(X)
x_test_final = Normalisation.gaussian(x_test_final)

y = np.ravel(y)

In [ ]:
boost = xgb.XGBRegressor(objective='reg:squarederror')

pipe = Pipeline(steps=[('regress', boost)])

# Model
max_depth = [3,4,5,6]
learning_rate = [0.05, 0.1, 0.15]
n_estimators = [200, 300]

parameters = dict(regress__learning_rate=learning_rate,
                    regress__n_estimators=n_estimators)

clf = GridSearchCV(pipe, parameters)
clf.fit(X, y)

# View The Best Parameters
print('learning_rate=', clf.best_estimator_.get_params()['regress__learning_rate'])
print('n_estimators=', clf.best_estimator_.get_params()['regress__n_estimators'])
best_learning_rate= clf.best_estimator_.get_params()['regress__learning_rate']
best_n_estimators= clf.best_estimator_.get_params()['regress__n_estimators']

## Final Prediction

In [ ]:
y_pred = clf.predict(x_test_final)
plt.hist(y_pred)
print("Train Score:", r2_score(y, clf.predict(X)))

y_pred_pd = pd.DataFrame(data=y_pred, columns=["y"])
y_pred_pd.to_csv('../Predictions/XGBoost_Optimised_Simple.csv', index_label='id')